<a href="https://colab.research.google.com/github/Alijgh003/My-Text2Speech/blob/master/MyTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 96.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 127.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 31.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 27.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 141.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.

In [ ]:
from TTS.api import TTS

In [ ]:
import torch

In [ ]:
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())


In [ ]:
# === 1) Knobs: set how you want it to behave ===
# Switch between raw text OR file upload by setting one of these

USE_UPLOAD         = True        # If True, you'll upload a .txt file below
RAW_TEXT           = """Paste a long text here if not using upload."""

OUTPUT_DIR         = "out4"       # Where paragraph files (and audio) will be saved
MAX_CHARS          = 5000        # Max characters per paragraph
MAX_SENTENCES      = 20           # Max sentences per paragraph (soft cap)
MAKE_AUDIO         = True       # If True, create one .wav per paragraph
MODEL_NAME         = "tts_models/multilingual/multi-dataset/your_tts"  # You can change it
LANGUAGE           = "en"        # For multilingual models (e.g., "en" for XTTS); otherwise None
TRY_TTS_SPLITTER   = True        # Try TTS built-in sentence splitter if available

# GPU usage for TTS inference (if audio enabled)
USE_GPU            = True        # Colab has a GPU runtime option


In [ ]:
# === 2) Utilities: upload file, splitting, and paragraph packing ===

import os, re, math
from pathlib import Path

# -- (A) Optional upload helper
text_source = None
if USE_UPLOAD:
    from google.colab import files  # Colab only
    up = files.upload()             # prompts for file(s)
    if up:
        fname = list(up.keys())[0]
        text_source = Path(fname).read_text(encoding="utf-8")
else:
    text_source = RAW_TEXT

if not text_source or not str(text_source).strip():
    raise ValueError("No input text provided (set USE_UPLOAD or fill RAW_TEXT).")

# -- (B) Fallback sentence splitter (regex)
def fallback_split_sentences(text: str):
    # Normalize whitespace and split on sentence enders . ! ?
    t = re.sub(r'\s+', ' ', text.strip())
    # Split at boundary after punctuation followed by space
    parts = [s.strip() for s in re.split(r'(?<=[.!?])\s+', t) if s.strip()]
    return parts if parts else [t]  # never return empty

# -- (C) Pack sentences into paragraphs by size/limits
def pack_sentences(sentences, max_chars=1200, max_sentences=4):
    paragraphs, buf, count, chars = [], [], 0, 0
    for s in sentences:
        add_len = len(s) + (1 if buf else 0)  # +1 space if not first in buf
        if (buf and (chars + add_len > max_chars)) or (count >= max_sentences):
            paragraphs.append(" ".join(buf))
            buf, count, chars = [s], 1, len(s)
        else:
            buf.append(s)
            count += 1
            chars += add_len
    if buf:
        paragraphs.append(" ".join(buf))
    return paragraphs


Saving 1.2410.12229v3.txt to 1.2410.12229v3 (1).txt


In [ ]:
text_source

'\n\nAbstract\nIn recent years, the introduction of knowledge graphs (KGs) has\nsignificantly advanced recommender systems by facilitating the discovery of potential associations between items. However, existing\nmethods still face several limitations. First, most KGs suffer from\nmissing facts or limited scopes. Second, existing methods convert\ntextual information in KGs into IDs, resulting in the loss of natural\nsemantic connections between different items. Third, existing methods struggle to capture high-order connections in the global KG.\nTo address these limitations, we propose a novel method called CoLaKG, which leverages large language models (LLMs) to improve\nKG-based recommendations. The extensive knowledge and remarkable reasoning capabilities of LLMs enable our method to supplement missing facts in KGs, and their powerful text understanding\nabilities allow for better utilization of semantic information. Specifically, CoLaKG extracts useful information from KGs at both l

In [ ]:
text_source = text_source.replace("Comprehending Knowledge Graphs with Large Language Models for Recommender Systems", "").replace(" SIGIR ’25, July 13–18, 2025, Padua, Italy", "").replace(" Ziqiang Cui, Yunpeng Weng, Xing Tang, Fuyuan Lyu, Dugang Liu, Xiuqiang He, and Chen Ma", "")

In [ ]:
text_source

'\n\nAbstract\nIn recent years, the introduction of knowledge graphs (KGs) has\nsignificantly advanced recommender systems by facilitating the discovery of potential associations between items. However, existing\nmethods still face several limitations. First, most KGs suffer from\nmissing facts or limited scopes. Second, existing methods convert\ntextual information in KGs into IDs, resulting in the loss of natural\nsemantic connections between different items. Third, existing methods struggle to capture high-order connections in the global KG.\nTo address these limitations, we propose a novel method called CoLaKG, which leverages large language models (LLMs) to improve\nKG-based recommendations. The extensive knowledge and remarkable reasoning capabilities of LLMs enable our method to supplement missing facts in KGs, and their powerful text understanding\nabilities allow for better utilization of semantic information. Specifically, CoLaKG extracts useful information from KGs at both l

In [ ]:
# === 3) Decide sentence splitter (TTS built-in vs regex) ===

sentences = None
tts = None
use_tts_splitter = False

if TRY_TTS_SPLITTER:
    try:
        from TTS.api import TTS as _TTS

        # Only create an instance if we need built-in splitting OR audio.
        # Warning: constructing may download a model.
        tts = _TTS(MODEL_NAME, gpu=USE_GPU and MAKE_AUDIO)
        # Try the built-in splitter if available
        if hasattr(tts, "split_text"):
            sentences = tts.split_text(text_source)
            use_tts_splitter = True
        else:
            sentences = fallback_split_sentences(text_source)
    except Exception as e:
        print(f"[Info] Built-in TTS splitter unavailable ({e}). Using regex fallback.")
        sentences = fallback_split_sentences(text_source)
else:
    sentences = fallback_split_sentences(text_source)

print(f"Splitter used: {'TTS built-in' if use_tts_splitter else 'Regex fallback'}")
print(f"Sentence count: {len(sentences)}")


/usr/local/lib/python3.11/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts


 99%|█████████▉| 422M/425M [00:19<00:00, 29.3MiB/s]

 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | 

In [ ]:
# === 4) Build paragraphs and save as para_XXX.txt ===

from pathlib import Path

out_dir = Path(OUTPUT_DIR)
out_dir.mkdir(parents=True, exist_ok=True)

paragraphs = pack_sentences(
    sentences,
    max_chars=MAX_CHARS,
    max_sentences=MAX_SENTENCES
)
width = max(3, int(math.log10(max(1, len(paragraphs))) + 1))

for i, para in enumerate(paragraphs, 1):
    (out_dir / f"para_{i:0{width}d}.txt").write_text(para, encoding="utf-8")

print(f"📝 Wrote {len(paragraphs)} paragraph files to {out_dir.resolve()}")


📝 Wrote 31 paragraph files to /content/out4


In [ ]:
paragraphs

['Abstract In recent years, the introduction of knowledge graphs (KGs) has significantly advanced recommender systems by facilitating the discovery of potential associations between items. However, existing methods still face several limitations. First, most KGs suffer from missing facts or limited scopes. Second, existing methods convert textual information in KGs into IDs, resulting in the loss of natural semantic connections between different items. Third, existing methods struggle to capture high-order connections in the global KG. To address these limitations, we propose a novel method called CoLaKG, which leverages large language models (LLMs) to improve KG-based recommendations. The extensive knowledge and remarkable reasoning capabilities of LLMs enable our method to supplement missing facts in KGs, and their powerful text understanding abilities allow for better utilization of semantic information. Specifically, CoLaKG extracts useful information from KGs at both local and glo

In [ ]:
# === 5) Optional: synthesize one WAV per paragraph (Coqui TTS) ===
# - Uses the same tts instance if already created, else creates one
# - Language is used for multilingual models (e.g., XTTS)

if MAKE_AUDIO:
    import torchaudio

    if tts is None:
        from TTS.api import TTS as _TTS
        tts = _TTS(MODEL_NAME, gpu=USE_GPU)

    for i, para in enumerate(paragraphs, 1):
        wav_path = out_dir / f"para_{i:0{width}d}.wav"
        kwargs = dict(text=para, file_path=str(wav_path), speaker_wav="john.mp3")
        if LANGUAGE:
            kwargs["language"] = LANGUAGE
        # Most builds split internally by default; if your model supports it, it will chunk long text safely
        tts.tts_to_file(**kwargs)

    print(f"🔊 Wrote {len(paragraphs)} audio files to {out_dir.resolve()}")


 > Text splitted to sentences.
['Abstract In recent years, the introduction of knowledge graphs (KGs) has significantly advanced recommender systems by facilitating the discovery of potential associations between items.', 'However, existing methods still face several limitations.', 'First, most KGs suffer from missing facts or limited scopes.', 'Second, existing methods convert textual information in KGs into IDs, resulting in the loss of natural semantic connections between different items.', 'Third, existing methods struggle to capture high-order connections in the global KG.', 'To address these limitations, we propose a novel method called CoLaKG, which leverages large language models (LLMs) to improve KG-based recommendations.', 'The extensive knowledge and remarkable reasoning capabilities of LLMs enable our method to supplement missing facts in KGs, and their powerful text understanding abilities allow for better utilization of semantic information.', 'Specifically, CoLaKG extrac

In [ ]:
# === Merge paragraph WAVs into a single file ===
# - Looks for: OUTPUT_DIR/para_*.wav
# - Concatenates in filename order
# - Resamples and converts to mono if needed
# - Writes: OUTPUT_DIR/final_merged.wav

import math
import torch
import torchaudio
from torchaudio.functional import resample
from pathlib import Path

# Folder where your paragraph WAVs were saved
FINAL_NAME = "final_merged.wav"    # output file name
TARGET_CHANNELS = 1                # force mono
SILENCE_SECS = 0.0                 # optional pause between clips (set e.g. 0.2)

out_dir = Path(OUTPUT_DIR)
wav_paths = sorted(out_dir.glob("para_*.wav"))

if not wav_paths:
    raise FileNotFoundError(f"No paragraph WAVs found in {out_dir} (expected para_XXX.wav).")

# Load first file to establish target sample rate
first_waveform, first_sr = torchaudio.load(str(wav_paths[0]))  # shape: [C, T]
target_sr = int(first_sr)

# Helper: make mono
def to_mono(wf: torch.Tensor) -> torch.Tensor:
    # wf: [C, T]
    if wf.size(0) == 1:
        return wf
    return wf.mean(dim=0, keepdim=True)

# Helper: (optional) insert silence between clips
def make_silence(seconds: float, sr: int, channels: int = 1) -> torch.Tensor:
    if seconds <= 0:
        return torch.zeros((channels, 0), dtype=torch.float32)
    num_samples = int(round(seconds * sr))
    return torch.zeros((channels, num_samples), dtype=torch.float32)

clips = []

for p in wav_paths:
    wf, sr = torchaudio.load(str(p))  # [C, T], sr
    # Convert to mono if needed
    wf = to_mono(wf)

    # Resample if needed
    if sr != target_sr:
        wf = resample(wf, orig_freq=sr, new_freq=target_sr)

    clips.append(wf)
    # Optional silence between clips
    if SILENCE_SECS > 0:
        clips.append(make_silence(SILENCE_SECS, target_sr, TARGET_CHANNELS))

# Concatenate along time dimension
if len(clips) == 1:
    merged = clips[0]
else:
    merged = torch.cat(clips, dim=1)  # [1, total_T]

# Save final merged file
final_path = out_dir / FINAL_NAME
torchaudio.save(str(final_path), merged, target_sr)
print(f"✅ Merged {len(wav_paths)} files into: {final_path.resolve()}")
print(f"   Sample rate: {target_sr} Hz | Channels: {merged.size(0)} | Samples: {merged.size(1)}")


✅ Merged 31 files into: /content/out4/final_merged.wav
   Sample rate: 16000 Hz | Channels: 1 | Samples: 69603808


In [ ]:
!ls

'1.2410.12229v3 (1).txt'   1.2410.12229v3.txt   john.mp3   out4   sample_data
